In [ ]:
%load_ext autoreload
%autoreload 2

# Model multiplexer

Rework the linear model and wrap it into a model multiplexer, so that we can have one model per month-day.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import torch
import torch.distributions
import torch.nn as nn
import xarray as xr

from crims2s.dataset import S2SDataset, TransformedDataset
from crims2s.transform import CompositeTransform, AddBiweeklyDimTransform, AddMetadata, ExampleToPytorch
from crims2s.util import ECMWF_FORECASTS

In [ ]:
DATASET = '***BASEDIR***/mlready/2021-08-08-test/'

## make transform to interface dataset w/ linear model

In [ ]:
def std_estimator(dataset, dim=None):
    dataset_mean = dataset.mean(dim=dim)
    
    if dim is None:
        dim_sizes = [dataset.sizes[x] for x in dataset_mean.dims]
    elif isinstance(dim, str):
        dim_sizes = dataset.sizes[dim]
    else:
        dim_sizes = [dataset.sizes[x] for x in dim]
    
    n = np.prod(dim_sizes)
    
    return xr.ufuncs.sqrt(xr.ufuncs.square(dataset - dataset_mean).sum(dim=dim) / (n - 1))

In [ ]:
def model_to_distribution(model):   
    model_tp_mean = model.tp.isel(lead_time=-1).mean(dim='realization').rename('tp_mu')
    model_tp_std = std_estimator(model.tp.isel(lead_time=-1), dim='realization').rename('tp_sigma')
    
    model_t2m_mean = model.t2m.mean(dim=['lead_time', 'realization']).rename('t2m_mu')
    model_t2m_std = std_estimator(model.t2m, dim=['lead_time', 'realization']).rename('t2m_sigma')
    
    return xr.merge([
        model_tp_mean, model_tp_std, model_t2m_mean, model_t2m_std
    ]).drop('lead_time').rename(biweekly_forecast='lead_time')

In [ ]:
def obs_to_biweekly(obs):
    aggregate_obs_tp = obs.pr.sum(dim='lead_time', min_count=2).rename('tp')
    aggregate_obs_t2m = obs.t2m.mean(dim='lead_time')
    return xr.merge([aggregate_obs_tp, aggregate_obs_t2m])

In [ ]:
def linear_model_adapter(example):
    model = model_to_distribution(example['model'])
    obs = obs_to_biweekly(example['obs'])
    
    return {
        'model': model,
        'obs': obs
    }

In [ ]:
#raw_train_dataset = S2SDataset(DATASET, filter_str='0312.nc', include_features=False, years=list(range(2000,2017)))
#raw_val_dataset = S2SDataset(DATASET, filter_str='0312.nc', include_features=False, years=list(range(2017,2020)))

In [ ]:
filter_re = re.compile('01[0-9]{2}.nc$')

In [ ]:
raw_train_dataset = S2SDataset(DATASET, include_features=False, name_filter=lambda x: filter_re.search(x), years=list(range(2000,2017)))
raw_val_dataset = S2SDataset(DATASET, include_features=False, name_filter=lambda x: filter_re.search(x), years=list(range(2017,2020)))

In [ ]:
for k in raw_train_dataset[0]['model'].data_vars:
    print(k)

In [ ]:
raw_train_dataset[0]['model']['t2m']

In [ ]:
transform = CompositeTransform([AddBiweeklyDimTransform(), linear_model_adapter, AddMetadata(), ExampleToPytorch()])

train_dataset = TransformedDataset(raw_train_dataset, transform)
val_dataset = TransformedDataset(raw_val_dataset, transform)

In [ ]:
len(train_dataset)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, num_workers=4, batch_size=None, batch_sampler=None)
val_dataloader = torch.utils.data.DataLoader(val_dataset, num_workers=4, batch_size=None, batch_sampler=None)

In [ ]:
from typing import Union, Callable, Any, Hashable

In [ ]:
train_dataset[0]

In [ ]:
one_batch = next(iter(train_dataloader))

In [ ]:
class ModelMultiplexer(nn.Module):
    """Dispatch the training examples to multiple models depending on the example.
    For instance, we could use this to use a different model for every monthday forecast.
    
    Because it uses an arbitraty model for every sample, this module does not support batching.
    To use it, it is recommended to disable automatic batching on the dataloader."""
    def __init__(self, key, models):
        """Args:
            key: If a str, used as a key to fetch the model name from the example dict. 
                 If a callable, called on the example and should return to model name to use.
            models: A mapping from model names to model instances. They keys should correspond to what is returned when applying key on the example."""
        super().__init__()
        
        if isinstance(key, str):
            self.key_fn = lambda x: x[key]
        else:
            self.key_fn = key
            
        self.models = nn.ModuleDict(models)
        
    def forward(self, example):      
        model_name = self.key_fn(example)
        model = self.models[model_name]
        
        return model(example)
            

In [ ]:
class LinearModel(nn.Module):
    def __init__(self, *shape, fill_weights=0.0, fill_intercept=0.0):
        super().__init__()
        
        self.weights = nn.Parameter(torch.full(shape, fill_weights))
        self.intercept = nn.Parameter(torch.full(shape, fill_intercept))
        
    def forward(self, x):
        return self.intercept + self.weights * x + x

In [ ]:
class TempPrecipEMOS(nn.Module):
    def __init__(self, biweekly=False):
        super().__init__()
        
        shape = (3, 121, 240) if biweekly else (121, 240)
        
        self.tp_mu_model = LinearModel(*shape)
        self.tp_sigma_model = LinearModel(*shape, fill_intercept=1.0)
        
        self.t2m_mu_model = LinearModel(*shape)
        self.t2m_sigma_model = LinearModel(*shape, fill_intercept=1.0)
        
    def forward(self, example):
        forecast_tp_mu, forecast_tp_sigma = example['model_tp_mu'], example['model_tp_sigma']
        forecast_t2m_mu, forecast_t2m_sigma = example['model_t2m_mu'], example['model_t2m_sigma']
        
        tp_mu = self.tp_mu_model(forecast_tp_mu)
        tp_sigma = self.tp_sigma_model(forecast_tp_sigma)
        tp_sigma = torch.clip(tp_sigma, min=1e-6)

        t2m_mu = self.t2m_mu_model(forecast_t2m_mu)
        t2m_sigma = self.t2m_sigma_model(forecast_t2m_sigma)
        t2m_sigma = torch.clip(t2m_sigma, min=1e-6)
        
        tp_dist = torch.distributions.Normal(loc=tp_mu, scale=tp_sigma)
        t2m_dist = torch.distributions.Normal(loc=t2m_mu, scale=t2m_sigma)
        
        return t2m_dist, tp_dist

In [ ]:
monthdays = [f'{m:02}{d:02}' for m, d in ECMWF_FORECASTS]
weekly_models = {monthday: TempPrecipEMOS(biweekly=True) for monthday in monthdays}

In [ ]:
weekly_models

In [ ]:
len(weekly_models)

In [ ]:
monthdays[:5]

In [ ]:
model = ModelMultiplexer('monthday', weekly_models)

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)

In [ ]:
for epoch in range(20):
    train_losses = []
    train_temperature_losses = []
    train_rain_losses = []
    
    model.train()
    for example in train_dataloader:
        t2m_dist, tp_dist = model.forward(example)

        tp_obs = example['obs_tp']
        tp_nan_mask = tp_obs.isnan()
        tp_obs[tp_nan_mask] = 0.0
        tp_log_likelihood = tp_dist.log_prob(tp_obs)
        tp_log_likelihood[tp_nan_mask] = 0.0

        t2m_obs = example['obs_t2m']
        t2m_nan_mask = t2m_obs.isnan()
        t2m_obs[t2m_nan_mask] = 0.0
        t2m_log_likelihood = t2m_dist.log_prob(t2m_obs)
        t2m_log_likelihood[t2m_nan_mask] = 0.0

        rain_loss = -tp_log_likelihood.mean()
        temperature_loss =  -t2m_log_likelihood.mean()
        loss = rain_loss + temperature_loss
        
        loss.backward()
                
        optimizer.step()
        optimizer.zero_grad()
        
        train_losses.append(float(loss.detach()))
        train_temperature_losses.append(float(temperature_loss.detach()))
        train_rain_losses.append(float(rain_loss.detach()))

    train_mean_loss = np.array(train_losses).mean()
    train_mean_rain_loss = np.array(train_rain_losses).mean()
    train_mean_temperature_loss = np.array(train_temperature_losses).mean()
    print(f'Epoch {epoch} train loss: {train_mean_loss}. Temperature: {train_mean_temperature_loss}. Rain: {train_mean_rain_loss}.')
        
        
    model.eval()
    with torch.no_grad():
        val_losses = []
        val_rain_losses = []
        val_t2m_losses = []
        for example in val_dataloader:
            t2m_dist, tp_dist = model(example)
            
            obs_t2m, obs_tp = example['obs_t2m'], example['obs_tp']
            
            tp_obs = example['obs_tp']
            tp_nan_mask = tp_obs.isnan()
            tp_obs[tp_nan_mask] = 0.0
            tp_log_likelihood = tp_dist.log_prob(tp_obs)
            tp_log_likelihood[tp_nan_mask] = 0.0

            t2m_obs = example['obs_t2m']
            t2m_nan_mask = t2m_obs.isnan()
            t2m_obs[t2m_nan_mask] = 0.0
            t2m_log_likelihood = t2m_dist.log_prob(t2m_obs)
            t2m_log_likelihood[t2m_nan_mask] = 0.0
            
            val_rain_loss = -tp_log_likelihood.mean()
            val_temperature_loss =  -t2m_log_likelihood.mean()
            val_loss = val_rain_loss + val_temperature_loss
            
            val_rain_losses.append(val_rain_loss.detach())
            val_t2m_losses.append(val_temperature_loss.detach())
            val_losses.append(val_loss.detach())
        
        
    val_mean_loss = np.array(val_losses).mean()
    val_mean_rain_loss = np.array(val_rain_losses).mean()
    val_mean_temperature_loss = np.array(val_t2m_losses).mean()
    print(f'Epoch {epoch} val loss: {val_mean_loss}. Temperature: {val_mean_temperature_loss}. Rain: {val_mean_rain_loss}.')
    print()

In [ ]:
sns.lineplot(data=train_losses)

In [ ]:
val_rain_losses

In [ ]:
for k in model.models:
    print(k, model.models[k].tp_mu_model.weights[2][~t2m_nan_mask[0]].max())

In [ ]:
dicts = []

masks = {
    't2m': t2m_nan_mask[0],
    'tp': tp_nan_mask[0],
}

for monthday in monthdays[:4]:
    m = model.models[monthday]
    
    for parameter in ['mu', 'sigma']:
        for variable in ['t2m', 'tp']:
            attr_name = f'{variable}_{parameter}_model'
            linear_model = getattr(m, attr_name)
            mask = masks[variable]
            
            for component in ['weights', 'intercept']:
                model_component = getattr(linear_model, component)
                
                for lead_time in [0, 1, 2]:
                    at_lead_time = model_component[lead_time]
                    masked = at_lead_time[~mask].detach().numpy()
                
                    datapoints = [{
                        'monthday': monthday, 
                        'parameter': parameter,
                        'variable': variable,
                        'component': component,
                        'lead_time': lead_time,
                        'value': x
                    } for x in masked]
                    dicts.extend(datapoints)

In [ ]:
monthdays[:4]

In [ ]:
sns.histplot(data=masked)

In [ ]:
df = pd.DataFrame(dicts)

In [ ]:
df

In [ ]:
sns.set_theme()

In [ ]:
for variable in ['t2m', 'tp']:
    for parameter in ['mu', 'sigma']:
        for component in ['weights', 'intercept']:
            title = f'Distribution of {variable} {parameter} {component}'
            
            fig = plt.gcf()
            fig.set_size_inches(6,4)
            ax = sns.kdeplot(data=df[(df['component'] == component) & (df['variable'] == variable) & (df['parameter'] == parameter)], x='value', hue='lead_time')
            ax.set_title(title)
            plt.savefig(title.replace(' ', '_') + '.png')
            plt.show()
            

In [ ]:
sns.displot(data=df[(df['component'] == 'weights') & (df['variable'] == 't2m') & (df['parameter'] == 'mu')], x='value', row='lead_time').set_title('')

In [ ]:
sns.histplot(data=df[(df['parameter'] == 'mu') & (df['variable'] == 't2m') & (df['lead_time'] == 0) & (df['component'] == 'intercept')], x='value',hue='monthday')

In [ ]:
sns.histplot(data=model.models['0123'].tp_mu_model.intercept[2][~tp_nan_mask[0]].detach().numpy())

In [ ]:
sns.histplot(data=model.models['0123'].tp_mu_model.weights[1][~tp_nan_mask[0]].detach().numpy())

In [ ]:
sns.histplot(data=model.models['0123'].tp_sigma_model.intercept[1][~tp_nan_mask[0]].detach().numpy())

In [ ]:
sns.histplot(data=model.models['0123'].tp_sigma_model.weights[1][~tp_nan_mask[0]].detach().numpy())

In [ ]:
sns.histplot(data=model.models['0123'].t2m_sigma_model.weights[1][~t2m_nan_mask[0]].detach().numpy())

In [ ]:
sns.histplot(data=model.models['0123'].t2m_sigma_model.intercept[1][~t2m_nan_mask[0]].detach().numpy())

In [ ]:
sns.histplot(data=model.models['0123'].t2m_mu_model.intercept[1][~t2m_nan_mask[0]].detach().numpy())

In [ ]:
sns.histplot(data=model.models['0123'].t2m_mu_model.weights[1][~t2m_nan_mask[0]].detach().numpy())